In [3]:
import csv
import math
import sys
from datetime import datetime as dt
from datetime import timezone as tz

import numpy as np
import pandas as pd
import pytz as pytz
from keras.layers import Dense, Flatten
from keras.models import Sequential
from scipy.sparse import data
from sklearn.utils import validation
import tensorflow as tf
from tensorflow import keras
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.models import load_model

import utility
import pickle

In [5]:
def initDataset(inFileName, sourceCol):
    dataset = pd.read_csv(inFileName, header=0, infer_datetime_format=True, parse_dates=['UTC Time at End of Hour'], index_col=['UTC Time at End of Hour'])

    print(dataset.head())
    print(dataset.columns)
    dateTime = dataset.index.values
    
    print("\nAdding features related to date & time...")
    modifiedDataset = utility.addDateTimeFeatures(dataset, dateTime, sourceCol)
    dataset = modifiedDataset
    print("Features related to date & time added")
    
    for i in range(sourceCol, len(dataset.columns.values)):
        col = dataset.columns.values[i]
        dataset[col] = dataset[col].astype(np.float64)
        # print(col, dataset[col].dtype)

    return dataset, dateTime

In [6]:
# convert training data into inputs and outputs (labels)
def manipulateTrainingDataShape(data, trainWindowHours, labelWindowHours): 
    print("Data shape: ", data.shape)
    X, y = list(), list()
    # step over the entire history one time step at a time
    for i in range(len(data)-(trainWindowHours+labelWindowHours)+1):
        # define the end of the input sequence
        trainWindow = i + trainWindowHours
        labelWindow = trainWindow + labelWindowHours
        xInput = data[i:trainWindow, :]
        # xInput = xInput.reshape((len(xInput), 1))
        X.append(xInput)
        y.append(data[trainWindow:labelWindow, 0])
        # print(data[trainWindow:labelWindow, 0])
    return np.array(X, dtype=np.float64), np.array(y, dtype=np.float64)

def manipulateTestDataShape(data, slidingWindowLen, predictionWindowHours, isDates=False): 
    X = list()
    # step over the entire history one time step at a time
    for i in range(0, len(data)-(predictionWindowHours)+1, slidingWindowLen):
        # define the end of the input sequence
        predictionWindow = i + predictionWindowHours
        X.append(data[i:predictionWindow])
    if (isDates is False):
        X = np.array(X, dtype=np.float64)
    else:
        X = np.array(X)
    return X

In [31]:
CARBON_INTENSITY_COLUMN = 1
carbonRateDirect = {"avg_coal_production_forecast": 1003.7, "avg_biomass_production_forecast": 0, 
                "avg_nat_gas_production_forecast": 409.43, "avg_geothermal_production_forecast": 0, 
                "avg_hydro_production_forecast": 0, "avg_nuclear_production_forecast": 0, 
                "avg_oil_production_forecast": 406, "avg_solar_production_forecast": 0, 
                "avg_unknown_production_forecast": 575, "avg_others_production_forecast": 575, 
                "avg_wind_production_forecast": 0} # g/kWh

def calculateCarbonIntensity(dataset, carbonRate):
    global CARBON_INTENSITY_COLUMN
    carbonIntensity = 0
    carbonCol = []
    miniDataset = dataset.iloc[:, CARBON_INTENSITY_COLUMN:]
    print("**", miniDataset.columns.values)
    rowSum = miniDataset.sum(axis=1).to_list()
    for i in range(len(miniDataset)):
        if(rowSum[i] == 0):
            # basic algorithm to fill missing values if all sources are missing
            # just using the previous hour's value
            # same as electricityMap
            for j in range(1, len(dataset.columns.values)):
                if(dataset.iloc[i, j] == 0):
                    dataset.iloc[i, j] = dataset.iloc[i-1, j]
                miniDataset.iloc[i] = dataset.iloc[i, CARBON_INTENSITY_COLUMN:]
                # print(miniDataset.iloc[i])
            rowSum[i] = rowSum[i-1]
        carbonIntensity = 0
        for j in range(len(miniDataset.columns.values)):
            source = miniDataset.columns.values[j]
            sourceContribFrac = miniDataset.iloc[i, j]/rowSum[i]
            # print(sourceContribFrac, carbonRate[source])
            carbonIntensity += (sourceContribFrac * carbonRate[source])
        if (carbonIntensity == 0):
            print(miniDataset.iloc[i])
        carbonCol.append(round(carbonIntensity, 2)) # rounding to 2 values after decimal place
    dataset.insert(loc=CARBON_INTENSITY_COLUMN, column="carbon_intensity", value=carbonCol)
    return dataset

In [41]:
NUM_VAL_DAYS = 30
NUM_TEST_DAYS = 184
TRAINING_WINDOW_HOURS = 24
PREDICTION_WINDOW_HOURS = 24
MODEL_SLIDING_WINDOW_LEN = 24

def scale_dataset_trained(valData, testData, ftMin, ftMax):
    # Scaling columns to range (0, 1)
    row, col = valData.shape[0], valData.shape[1]
    for i in range(col):
        if((ftMax[i] - ftMin[i]) == 0):
            continue
        valData[:, i] = (valData[:, i] - ftMin[i]) / (ftMax[i] - ftMin[i])
        testData[:, i] = (testData[:, i] - ftMin[i]) / (ftMax[i] - ftMin[i])

    return valData, testData

def get_day_ahead_forecasts(model_filepath, history, train_window_hours, num_features, dep_var_column):
    global MODEL_SLIDING_WINDOW_LEN
    global PREDICTION_WINDOW_HOURS
    
    # Load the trained ANN model
    model = load_model(model_filepath)

    # walk-forward validation over each day
    print("Generating day-ahead forecasts...")
    predictions = []
    for i in range(len(history) // 24):
        day_ahead_predictions = []
        temp_history = history.copy()
        current_day_hours = i * MODEL_SLIDING_WINDOW_LEN
        for j in range(0, PREDICTION_WINDOW_HOURS, 24):
            # Get forecasts for the next 24 hours
            yhat_sequence, new_training_data = get_forecasts(model, temp_history, train_window_hours, num_features)
            day_ahead_predictions.extend(yhat_sequence)
            # Update history for predicting the next day
            latest_history = history[current_day_hours + j : current_day_hours + j + 24].copy()
            for k in range(24):
                latest_history[k][dep_var_column] = yhat_sequence[k]
            temp_history = np.concatenate([temp_history, latest_history], axis=0)

        # Update history for predicting the next day
        history = np.concatenate([history, history[current_day_hours : current_day_hours + MODEL_SLIDING_WINDOW_LEN]], axis=0)
        predictions.append(day_ahead_predictions)

    # Convert predictions to numpy array
    predicted_data = np.array(predictions, dtype=np.float64)
    return predicted_data


def get_forecasts(model, history, train_window_hours, num_features):
    # Flatten data
    data = np.array(history, dtype=np.float64)
    # Retrieve last observations for input data
    input_x = data[-train_window_hours:]
    # Reshape into [1, n_input, num_features]
    input_x = input_x.reshape((1, len(input_x), num_features))
    # Make predictions
    yhat = model.predict(input_x, verbose=0)
    # Extract the vector forecast
    yhat = yhat[0]
    return yhat, input_x

def inference_test(fuel_sources, inference_timestamp):
    for source in fuel_sources:
        IN_FILE_NAME = "data/MW_electricity_cleaned.csv"
        IN_MODEL_NAME = 'model/' + source + "_ann.keras"

        NUM_FEATURES_DICT = {"coal": 6, "nat_gas": 6, "nuclear": 6, "oil": 6, "hydro": 6, "solar": 6,
                            "wind": 6, "others": 6}

        NUM_VAL_DAYS = 30
        NUM_TEST_DAYS = 184
        TRAINING_WINDOW_HOURS = 24
        MODEL_SLIDING_WINDOW_LEN = 24
        PREDICTION_WINDOW_HOURS = 24

        COAL = 1
        NAT_GAS = 2
        NUCLEAR = 3
        OIL = 4
        HYDRO = 5
        SOLAR = 6
        WIND = 7
        OTHERS = 8

        FUEL = {COAL: "coal", NAT_GAS: "nat_gas", NUCLEAR: "nuclear", OIL: "oil", HYDRO: "hydro", SOLAR: "solar", WIND: "wind", OTHERS: "others"}
        SOURCE_TO_SOURCE_COL_MAP = {y: x for x, y in FUEL.items()}

        SOURCE_COL = SOURCE_TO_SOURCE_COL_MAP[source]
        NUM_FEATURES = NUM_FEATURES_DICT[FUEL[SOURCE_COL]]

        print("initializing dataset...")
        
        dataset, dateTime = initDataset(IN_FILE_NAME, SOURCE_COL)
        nearest_lower_timestamp = max(filter(lambda x: x <= np.datetime64(inference_timestamp), dateTime))

        # Get data up to last_date and last 24 hours of data
        last_past_date = pd.to_datetime(nearest_lower_timestamp).strftime("%Y-%m-%d %H:%M:%S")
        past = dataset.loc[dataset.index <= last_past_date].tail(24)

        # Get data minimum last_date, max last_date + 24 hours of data
        last_future_date = (pd.to_datetime(nearest_lower_timestamp) + pd.Timedelta(hours=24)).strftime("%Y-%m-%d %H:%M:%S")
        future = dataset.loc[dataset.index <= last_future_date].tail(24)

        # trainData, valData, testData, fullTrainData = utility.splitDataset(dataset.values, NUM_TEST_DAYS, NUM_VAL_DAYS)

        # trainDates = dateTime[: -(NUM_TEST_DAYS*24)]
        # fullTrainDates = np.copy(trainDates)
        # trainDates, validationDates = trainDates[: -(NUM_VAL_DAYS*24)], trainDates[-(NUM_VAL_DAYS*24):]
        # testDates = dateTime[-(NUM_TEST_DAYS*24):]

        trainDates = dateTime[: -(NUM_TEST_DAYS*24)]
        fullTrainDates = np.copy(trainDates)
        trainDates, validationDates = trainDates[: -(NUM_VAL_DAYS*24)], trainDates[-(NUM_VAL_DAYS*24):]
        testDates = future.index

        past = past.iloc[:, SOURCE_COL: SOURCE_COL+NUM_FEATURES].values
        future = future.iloc[:, SOURCE_COL: SOURCE_COL+NUM_FEATURES].values

        print("past shape: ", past.shape) # (days x hour) x features
        print("future shape: ", future.shape) # (days x hour) x features

        for i in range(past.shape[0]):
            for j in range(past.shape[1]):
                if(np.isnan(past[i, j])):
                    past[i, j] = past[i-1, j]

        for i in range(future.shape[0]):
            for j in range(future.shape[1]):
                if(np.isnan(future[i, j])):
                    future[i, j] = future[i-1, j]

        featureList = dataset.columns.values[SOURCE_COL:SOURCE_COL+NUM_FEATURES]
        print("Features: ", featureList)

        data = []
        with open(f'../GhostPostCC/model/{source}_ann.pkl', 'rb') as f:
            while True:
                try:
                    data.append(pickle.load(f))
                except EOFError:
                    break
        
        model = data[0]
        ftMin = data[1]
        ftMax = data[2]

        print("Scaling data...")
        past, future = scale_dataset_trained(past, future, ftMin, ftMax)
        print("***** Data scaling done *****")
        print(past.shape, future.shape)

        history = past.tolist()
        predictedData = get_day_ahead_forecasts(IN_MODEL_NAME, history, TRAINING_WINDOW_HOURS, NUM_FEATURES, 0)
        actualData = manipulateTestDataShape(future[:, 0], MODEL_SLIDING_WINDOW_LEN, PREDICTION_WINDOW_HOURS, False)
        
        formattedTestDates = manipulateTestDataShape(testDates, MODEL_SLIDING_WINDOW_LEN, PREDICTION_WINDOW_HOURS, True)
        formattedTestDates = np.reshape(formattedTestDates, formattedTestDates.shape[0]*formattedTestDates.shape[1])
        actualData = actualData.astype(np.float64)

        print("ActualData shape: ", actualData.shape)
        actual = np.reshape(actualData, actualData.shape[0]*actualData.shape[1])
        print("actual.shape: ", actual.shape)
        unscaledTestData = utility.inverseDataScaling(actual, ftMax[0], ftMin[0])
        predictedData = predictedData.astype(np.float64)
        print("PredictedData shape: ", predictedData.shape)
        predicted = np.reshape(predictedData, predictedData.shape[0]*predictedData.shape[1])
        print("predicted.shape: ", predicted.shape)
        unScaledPredictedData = utility.inverseDataScaling(predicted, ftMax[0], ftMin[0])
        rmseScore, mapeScore = utility.getScores(actualData, predictedData, unscaledTestData, unScaledPredictedData)
        print("***** Forecast done *****")
        print("Overall RMSE score: ", rmseScore)
        print(rmseScore)

        data = []
        for i in range(len(unScaledPredictedData)):
                row = []
                row.append(str(formattedTestDates[i]))
                row.append(str(unscaledTestData[i]))
                row.append(str(unScaledPredictedData[i]))
                data.append(row)
    return data

def inference(fuel_sources, inference_timestamp):
    combined_data = pd.DataFrame(columns=["timestamp"])
    for source in fuel_sources:
        IN_FILE_NAME = "data/MW_electricity_cleaned.csv"
        IN_MODEL_NAME = 'model/' + source + "_ann.keras"

        NUM_FEATURES_DICT = {"coal": 6, "nat_gas": 6, "nuclear": 6, "oil": 6, "hydro": 6, "solar": 6,
                            "wind": 6, "others": 6}

        NUM_VAL_DAYS = 30
        NUM_TEST_DAYS = 184
        TRAINING_WINDOW_HOURS = 24
        PREDICTION_WINDOW_HOURS = 24
        MODEL_SLIDING_WINDOW_LEN = 24

        COAL = 1
        NAT_GAS = 2
        NUCLEAR = 3
        OIL = 4
        HYDRO = 5
        SOLAR = 6
        WIND = 7
        OTHERS = 8

        FUEL = {COAL: "coal", NAT_GAS: "nat_gas", NUCLEAR: "nuclear", OIL: "oil", HYDRO: "hydro", SOLAR: "solar", WIND: "wind", OTHERS: "others"}
        SOURCE_TO_SOURCE_COL_MAP = {y: x for x, y in FUEL.items()}

        SOURCE_COL = SOURCE_TO_SOURCE_COL_MAP[source]
        NUM_FEATURES = NUM_FEATURES_DICT[FUEL[SOURCE_COL]]

        print("initializing dataset...")
        
        dataset, dateTime = initDataset(IN_FILE_NAME, SOURCE_COL)
        nearest_lower_timestamp = max(filter(lambda x: x <= np.datetime64(inference_timestamp), dateTime))

        # Get data up to last_date and last 24 hours of data
        last_past_date = pd.to_datetime(nearest_lower_timestamp).strftime("%Y-%m-%d %H:%M:%S")
        past = dataset.loc[dataset.index <= last_past_date].tail(24)

        # Get data minimum last_date, max last_date + 24 hours of data
        last_future_date = (pd.to_datetime(nearest_lower_timestamp) + pd.Timedelta(hours=24)).strftime("%Y-%m-%d %H:%M:%S")
        future = dataset.loc[dataset.index <= last_future_date].tail(24)

        trainDates = dateTime[: -(NUM_TEST_DAYS*24)]
        fullTrainDates = np.copy(trainDates)
        trainDates, validationDates = trainDates[: -(NUM_VAL_DAYS*24)], trainDates[-(NUM_VAL_DAYS*24):]
        testDates = future.index

        past = past.iloc[:, SOURCE_COL: SOURCE_COL+NUM_FEATURES].values
        future = future.iloc[:, SOURCE_COL: SOURCE_COL+NUM_FEATURES].values

        print("past shape: ", past.shape) # (days x hour) x features
        print("future shape: ", future.shape) # (days x hour) x features

        for i in range(past.shape[0]):
            for j in range(past.shape[1]):
                if(np.isnan(past[i, j])):
                    past[i, j] = past[i-1, j]

        featureList = dataset.columns.values[SOURCE_COL:SOURCE_COL+NUM_FEATURES]
        print("Features: ", featureList)

        data = []
        with open(f'../GhostPostCC/model/{source}_ann.pkl', 'rb') as f:
            while True:
                try:
                    data.append(pickle.load(f))
                except EOFError:
                    break
        
        model = data[0]
        ftMin = data[1]
        ftMax = data[2]

        print("Scaling data...")
        past, future = scale_dataset_trained(past, future, ftMin, ftMax)
        print("***** Data scaling done *****")
        print(past.shape, future.shape)

        history = past.tolist()
        predictedData = get_day_ahead_forecasts(IN_MODEL_NAME, history, TRAINING_WINDOW_HOURS, NUM_FEATURES, 0)
        formattedTestDates = manipulateTestDataShape(testDates, MODEL_SLIDING_WINDOW_LEN, PREDICTION_WINDOW_HOURS, True)
        formattedTestDates = np.reshape(formattedTestDates, formattedTestDates.shape[0]*formattedTestDates.shape[1])

        predictedData = predictedData.astype(np.float64)
        print("PredictedData shape: ", predictedData.shape)
        predicted = np.reshape(predictedData, predictedData.shape[0]*predictedData.shape[1])
        print("predicted.shape: ", predicted.shape)
        unScaledPredictedData = utility.inverseDataScaling(predicted, ftMax[0], ftMin[0])

        df = pd.DataFrame(unScaledPredictedData, columns=[f"avg_{source}_production_forecast"])
        combined_data = pd.concat([combined_data, df], axis=1)
    combined_data["timestamp"] = pd.to_datetime(formattedTestDates).strftime("%Y-%m-%d %H:%M:%S")

    combined_data = calculateCarbonIntensity(combined_data, carbonRateDirect)

    return combined_data

In [42]:
aa = inference(['coal', 'hydro', 'wind'], '2023-04-07 10:02:00')

initializing dataset...
                         Unnamed: 0  Net Generation (MW) from Coal  \
UTC Time at End of Hour                                              
2021-01-01 06:00:00               0                        25367.0   
2021-01-01 07:00:00               1                        24662.0   
2021-01-01 08:00:00               2                        25365.0   
2021-01-01 09:00:00               3                        24811.0   
2021-01-01 10:00:00               4                        25041.0   

                         Net Generation (MW) from Natural Gas  \
UTC Time at End of Hour                                         
2021-01-01 06:00:00                                   15999.0   
2021-01-01 07:00:00                                   16063.0   
2021-01-01 08:00:00                                   16057.0   
2021-01-01 09:00:00                                   15870.0   
2021-01-01 10:00:00                                   15755.0   

                         Net 

/var/folders/fh/7sz134vx5jq6_78w16smcs3h0000gn/T/ipykernel_92116/2017388480.py:2: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  dataset = pd.read_csv(inFileName, header=0, infer_datetime_format=True, parse_dates=['UTC Time at End of Hour'], index_col=['UTC Time at End of Hour'])


18738 7536
                         Unnamed: 0  Net Generation (MW) from Coal  hour_sin  \
UTC Time at End of Hour                                                        
2021-01-01 06:00:00               0                        25367.0  1.000000   
2021-01-01 07:00:00               1                        24662.0  0.965926   
2021-01-01 08:00:00               2                        25365.0  0.866025   
2021-01-01 09:00:00               3                        24811.0  0.707107   
2021-01-01 10:00:00               4                        25041.0  0.500000   

                             hour_cos  month_sin  month_cos  weekend  \
UTC Time at End of Hour                                                
2021-01-01 06:00:00      6.123234e-17   0.008601   0.999963        0   
2021-01-01 07:00:00     -2.588190e-01   0.009318   0.999957        0   
2021-01-01 08:00:00     -5.000000e-01   0.010035   0.999950        0   
2021-01-01 09:00:00     -7.071068e-01   0.010751   0.999942        0

/var/folders/fh/7sz134vx5jq6_78w16smcs3h0000gn/T/ipykernel_92116/2017388480.py:2: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  dataset = pd.read_csv(inFileName, header=0, infer_datetime_format=True, parse_dates=['UTC Time at End of Hour'], index_col=['UTC Time at End of Hour'])


18738 7536
                         Unnamed: 0  Net Generation (MW) from Coal  \
UTC Time at End of Hour                                              
2021-01-01 06:00:00               0                        25367.0   
2021-01-01 07:00:00               1                        24662.0   
2021-01-01 08:00:00               2                        25365.0   
2021-01-01 09:00:00               3                        24811.0   
2021-01-01 10:00:00               4                        25041.0   

                         Net Generation (MW) from Natural Gas  \
UTC Time at End of Hour                                         
2021-01-01 06:00:00                                   15999.0   
2021-01-01 07:00:00                                   16063.0   
2021-01-01 08:00:00                                   16057.0   
2021-01-01 09:00:00                                   15870.0   
2021-01-01 10:00:00                                   15755.0   

                         Net Generation (M

/var/folders/fh/7sz134vx5jq6_78w16smcs3h0000gn/T/ipykernel_92116/2017388480.py:2: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  dataset = pd.read_csv(inFileName, header=0, infer_datetime_format=True, parse_dates=['UTC Time at End of Hour'], index_col=['UTC Time at End of Hour'])


                         Unnamed: 0  Net Generation (MW) from Coal  \
UTC Time at End of Hour                                              
2021-01-01 06:00:00               0                        25367.0   
2021-01-01 07:00:00               1                        24662.0   
2021-01-01 08:00:00               2                        25365.0   
2021-01-01 09:00:00               3                        24811.0   
2021-01-01 10:00:00               4                        25041.0   

                         Net Generation (MW) from Natural Gas  \
UTC Time at End of Hour                                         
2021-01-01 06:00:00                                   15999.0   
2021-01-01 07:00:00                                   16063.0   
2021-01-01 08:00:00                                   16057.0   
2021-01-01 09:00:00                                   15870.0   
2021-01-01 10:00:00                                   15755.0   

                         Net Generation (MW) from Nuc

In [44]:
aa.to_json()

'{"timestamp":{"0":"2023-04-07 11:00:00","1":"2023-04-07 12:00:00","2":"2023-04-07 13:00:00","3":"2023-04-07 14:00:00","4":"2023-04-07 15:00:00","5":"2023-04-07 16:00:00","6":"2023-04-07 17:00:00","7":"2023-04-07 18:00:00","8":"2023-04-07 19:00:00","9":"2023-04-07 20:00:00","10":"2023-04-07 21:00:00","11":"2023-04-07 22:00:00","12":"2023-04-07 23:00:00","13":"2023-04-08 00:00:00","14":"2023-04-08 01:00:00","15":"2023-04-08 02:00:00","16":"2023-04-08 03:00:00","17":"2023-04-08 04:00:00","18":"2023-04-08 05:00:00","19":"2023-04-08 06:00:00","20":"2023-04-08 07:00:00","21":"2023-04-08 08:00:00","22":"2023-04-08 09:00:00","23":"2023-04-08 10:00:00"},"carbon_intensity":{"0":466.9,"1":478.08,"2":487.38,"3":502.98,"4":524.82,"5":538.1,"6":555.51,"7":550.27,"8":538.41,"9":542.22,"10":542.23,"11":531.7,"12":551.09,"13":533.39,"14":543.22,"15":531.26,"16":547.65,"17":542.15,"18":538.96,"19":518.45,"20":514.09,"21":495.33,"22":481.02,"23":481.31},"avg_coal_production_forecast":{"0":12686.60027,"1